In [2]:
! pip install transformers huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [3]:
import os
HF_HUB = "<huggingface key>"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_HUB

In [4]:
from langchain import HuggingFaceHub

model_name = "tiiuae/falcon-7b"
model_name = "lmsys/fastchat-t5-3b-v1.0"

model_kwargs = {
    "temperature": 0.9,
    "max_length": 1024
}

llm = HuggingFaceHub(
    repo_id = model_name,
    model_kwargs = model_kwargs
)

In [6]:
prompt = "What is the name of orange fruit"
llm(prompt)

'<pad> orange'

In [7]:
# Prompt Templating
from langchain import PromptTemplate
template = "You are naming consultant for new companies. What is the good name for the company that makes {product}"
prompt = PromptTemplate.from_template(template)
prompt.format(product="CCTV")

'You are naming consultant for new companies. What is the good name for the company that makes CCTV'

In [8]:
from langchain.chains import LLMChain
template = "You are naming consultant for new companies. What is the good name for the {company} that makes {product}"
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
chain.run(
    {
        "company": "Arcane Eye",
        "product": "Colorful Socks"
    }
)

'<pad> "Socksense"\n'

In [10]:
# Sequential Chains
from langchain.chains import SimpleSequentialChain
template = "What is the good name for a company that makes {product}"

first_prompt = PromptTemplate.from_template(template)
first_chain = LLMChain(llm=llm, prompt=first_prompt)
first_chain.run("colorful socks")

'<pad> Socks  Inc.\n'

In [11]:
second_template = "Write a catch phrase for the following company: {company_name}"
second_prompt = PromptTemplate.from_template(second_template)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [12]:
overall_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)
catch_phrase = overall_chain.run("colorful socks")



> Entering new SimpleSequentialChain chain...
<pad> Socks  Inc.

<pad> "Socks  that  keep  you  going  strong.  Socks  Inc."


> Finished chain.


In [13]:

# Agents

! pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=5deae8c195711ea6abd7ec4ab7753d16ba32e33912dfac80a5da044ca208526e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [14]:
import pprint
from langchain.agents import get_all_tool_names, load_tools, initialize_agent
prompt = "When was the 3rd president of the US born? What is the year raised to power of 3"
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(get_all_tool_names())


[   'python_repl',
    'requests',
    'requests_get',
    'requests_post',
    'requests_patch',
    'requests_put',
    'requests_delete',
    'terminal',
    'sleep',
    'wolfram-alpha',
    'google-search',
    'google-search-results-json',
    'searx-search-results-json',
    'bing-search',
    'metaphor-search',
    'ddg-search',
    'google-serper',
    'google-serper-results-json',
    'serpapi',
    'dalle-image-generator',
    'twilio',
    'searx-search',
    'wikipedia',
    'arxiv',
    'golden-query',
    'pubmed',
    'human',
    'awslambda',
    'sceneXplain',
    'graphql',
    'openweathermap-api',
    'dataforseo-api-search',
    'dataforseo-api-search-json',
    'news-api',
    'tmdb-api',
    'podcast-api',
    'llm-math',
    'open-meteo-api']


In [15]:
tools = load_tools(
    ["wikipedia", "llm-math"],
    llm=llm
)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [16]:
agent.run(prompt)



> Entering new AgentExecutor chain...
<pad>  you  should  always  think  about  what  to  do
 Action:  you  should  use  the  calculator  to  find  the  answer
 Action  Input:  the  input  to  the  action
 Observation:  the  result  of  the  action
 Thought:  I  now  know  the  final  answer
 Final  Answer:  the  final  answer  to  the  original  input  question

Observation: you  should  use  the  calculator  to  find  the  answer is not a valid tool, try one of [Wikipedia, Calculator].
Thought:<pad>  you  should  always  think  about  what  to  do
 Action:  you  should  use  the  Wikipedia  to  find  the  answer
 Action
 Input:  the  input  to  the  action
 Observation:  the  result  of  the  action
 Thought:  I  now  know  the  final  answer
 Final  Answer:  the  final  answer  to  the  original  input  question
 Observation:  you  should  use  the  Wikipedia  to  find  the  answer  is  not  a  valid  tool,  try  one  of  [Wikipedia,  Calculator].

Observation: you  should  use  t

ValueError: ignored

In [ ]:
# Human Agent
from langchain.agents import AgentType
tools = load_tools(['human'])

In [ ]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_chain.run("What is my friends name")